- Given a lat lon
- Assume maximum temperature
- convert lat lon to correct projection
- join all times together
- from megacube extract that geospatial point
    - investgate interopolation vs. find nearest
- collapse time by mean
- return float

In [19]:
REQUEST = '''{
    "args": {
        "date": ["2013-02-27"],
        "start_date": ["2013-01-27"],
        "end_date": ["2013-02-27"],
        "lat": ["52"],
        "lon": ["-1"]
    },
    "path": {
        "parameter": "temperature"
    }
}'''

In [20]:
# GET /helloworld

print('hello world')


hello world


In [21]:
import datetime
import iris

iris.FUTURE.netcdf_promote = True

def load_data(pattern):
    # pattern is glob style
    cubes = iris.load('/opt/data/{}'.format(pattern))
    return iris.cube.CubeList(cubes).concatenate_cube()


In [22]:
import cartopy

def transform_coords(cube, lon, lat):
    expected_proj = cube.coords('projection_y_coordinate')[0].coord_system.as_cartopy_crs()
    given_proj = cartopy.crs.PlateCarree()
    return expected_proj.transform_point(lon, lat, given_proj)

In [23]:

def get_coords(cube, lat, lon):
    x, y = transform_coords(cube, lon, lat)
    first_year = next(cube.slices_over('time'))
    samples = [('projection_y_coordinate', y), ('projection_x_coordinate', x)]
    first_point = first_year.interpolate(samples, iris.analysis.Nearest())
    exact_x = first_point.coord('projection_x_coordinate')[0].points[0]
    exact_y = first_point.coord('projection_y_coordinate')[0].points[0]
    
    return(exact_x, exact_y)
    


In [24]:

def collapse_latlon(cube, x, y):
    x_const = iris.Constraint(projection_x_coordinate=x)
    y_const = iris.Constraint(projection_y_coordinate=y)
    return cube.extract(x_const).extract(y_const)



In [25]:
from datetime import timedelta

def expand_years(end_date, past_years=10):
    years = [
        datetime.datetime(
            end_date.year - i,
            end_date.month,
            end_date.day)
        for i in range(past_years)]
    return years

def select_all_days(start, end):
    delta = end - start
    days = [end - timedelta(days=i) for i in range(delta.days)]
    return days

def extract_dates(cube, dates):
    time_units = cube.coord('time').units
    vals = [time_units.date2num(date) for date in dates]
    time_constraint = iris.Constraint(time=vals)
    return cube.extract(time_constraint)

In [26]:

def mean(cube):
    return cube.collapsed('time', iris.analysis.MEAN)


In [27]:
patterns = {
    'temperature': 'maximum-temperature/',
    'rainfall': 'rainfall/'
}

In [28]:
def get_pattern(parameter, dates):
    base = patterns[parameter]
    years = set([date.year for date in dates])
    pattern = '*_('+'|'.join([str(y) for y in years])+')*.nc'
    return base + pattern

In [38]:
from collections import defaultdict
CACHED_POINTS = defaultdict(lambda: None)

In [39]:
def get_response(cube, dates, x, y):
    time_cube = collapse_latlon(cube, x, y)
    collapsed_time_cube = extract_dates(time_cube, dates)
    
    val = float(mean(collapsed_time_cube).data)
    
    response = {
        'value': val,
        'operation': 'mean',
        'start_date': dates[-1].strftime('%Y-%m-%d'),
        'end_date': dates[0].strftime('%Y-%m-%d')
    }
    return response

In [42]:
# GET /:parameter/mean/climatology
import json

req = json.loads(REQUEST)

args = req['args']
lat = float(args['lat'][0])
lon = float(args['lon'][0])

if 'date' in args.keys():
    date = datetime.datetime.strptime(args['date'][0], '%Y-%m-%d')
else:
    date = datetime.datetime.now()

path = req['path']
param = path['parameter']

dates = expand_years(date)
data_path = get_pattern(param, dates)
cube = load_data(data_path)
x, y = get_coords(cube, lat, lon)

cache_key = ('climatology', param, x, y)
if CACHED_POINTS[cache_key] != None:
    response = CACHED_POINTS[cache_key]
else:
    response = get_response(cube, dates, x, y)
    CACHED_POINTS[cache_key] = response

print(json.dumps(response))

{"end_date": "2013-02-27", "operation": "mean", "value": 7.932449066638947, "start_date": "2004-02-27"}


/Users/alex/anaconda3/lib/python3.5/site-packages/iris/coords.py:992: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))


In [ ]:
# GET /:parameter/mean/range
import json

req = json.loads(REQUEST)

args = req['args']
lat = float(args['lat'][0])
lon = float(args['lon'][0])

start_date = datetime.datetime.strptime(args['start_date'][0], '%Y-%m-%d')
end_date = datetime.datetime.strptime(args['end_date'][0], '%Y-%m-%d')

path = req['path']
param = path['parameter']
data_path = patterns[param]
cube = load_data(data_path)
x, y = get_coords(cube, lat, lon)

cache_key = ('range', param, x, y)

if CACHED_POINTS[cache_key] != None:
    response = CACHED_POINTS[cache_key]
else:
    dates = select_all_days(start_date, end_date)
    response = get_response(cube, dates, x, y)
    CACHED_POINTS[cache_key] = response

print(json.dumps(response))

In [24]:
# ResponseInfo GET /:parameter/mean/range
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    }
}))

NameError: name 'json' is not defined

In [ ]:
# ResponseInfo GET /:parameter/mean/climatology
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    }
}))